<a href="https://colab.research.google.com/github/Anantika0410/Mini-Project/blob/main/test4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install wfdb and compatible pandas
!pip install pandas==2.2.3 wfdb --upgrade --quiet

# Download normal and abnormal ECG files
!wget -q -O 100.dat https://physionet.org/files/mitdb/1.0.0/100.dat?download
!wget -q -O 100.hea https://physionet.org/files/mitdb/1.0.0/100.hea?download

!wget -q -O 105.dat https://physionet.org/files/mitdb/1.0.0/105.dat?download
!wget -q -O 105.hea https://physionet.org/files/mitdb/1.0.0/105.hea?download


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 10.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.


In [ ]:
import wfdb
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# Load records
record_normal = wfdb.rdrecord('100')
record_abnormal = wfdb.rdrecord('105')

# Take channel 0
signal_normal = record_normal.p_signal[:, 0]
signal_abnormal = record_abnormal.p_signal[:, 0]

# Split into windows
window_size = 140

def split_into_windows(signal, label):
    X, y = [], []
    for i in range(0, len(signal) - window_size, window_size):
        segment = signal[i:i+window_size]
        X.append(segment)
        y.append(label)
    return np.array(X), np.array(y)

X_normal, y_normal = split_into_windows(signal_normal, 0)
X_abnormal, y_abnormal = split_into_windows(signal_abnormal, 1)

# Combine
X = np.concatenate((X_normal, X_abnormal), axis=0)
y = np.concatenate((y_normal, y_abnormal), axis=0)

# Shuffle the data
indices = np.arange(X.shape[0])
np.random.shuffle(indices)
X = X[indices]
y = y[indices]

# Reshape for LSTM (samples, timesteps, features)
X = X.reshape((X.shape[0], X.shape[1], 1))

# Split into train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Training samples: {X_train.shape}, Testing samples: {X_test.shape}")


Training samples: (7427, 140, 1), Testing samples: (1857, 140, 1)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Dropout, Flatten

# Build CNN-LSTM hybrid model
model = Sequential()

# 1D CNN layers to extract features
model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(140, 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.3))

model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.3))

# LSTM layers to learn sequence
model.add(LSTM(64, return_sequences=False))  # No return sequences now
model.add(Dropout(0.3))

# Output layer
model.add(Dense(1, activation='sigmoid'))  # Binary classification

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 138, 32)        │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 69, 32)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 69, 32)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 67, 64)         │         6,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 33, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 33, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 64)             │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 39,425 (154.00 KB)

 Trainable params: 39,425 (154.00 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
history = model.fit(X_train, y_train, epochs=30, batch_size=64, validation_split=0.2)


Epoch 1/30
93/93 ━━━━━━━━━━━━━━━━━━━━ 9s 55ms/step - accuracy: 0.6265 - loss: 0.6166 - val_accuracy: 0.7456 - val_loss: 0.5032
Epoch 2/30
93/93 ━━━━━━━━━━━━━━━━━━━━ 4s 42ms/step - accuracy: 0.7959 - loss: 0.4330 - val_accuracy: 0.8910 - val_loss: 0.2636
Epoch 3/30
93/93 ━━━━━━━━━━━━━━━━━━━━ 6s 56ms/step - accuracy: 0.9045 - loss: 0.2511 - val_accuracy: 0.9213 - val_loss: 0.2043
Epoch 4/30
93/93 ━━━━━━━━━━━━━━━━━━━━ 9s 43ms/step - accuracy: 0.9426 - loss: 0.1724 - val_accuracy: 0.9576 - val_loss: 0.1340
Epoch 5/30
93/93 ━━━━━━━━━━━━━━━━━━━━ 5s 55ms/step - accuracy: 0.9550 - loss: 0.1400 - val_accuracy: 0.9596 - val_loss: 0.1219
Epoch 6/30
93/93 ━━━━━━━━━━━━━━━━━━━━ 10s 51ms/step - accuracy: 0.9576 - loss: 0.1246 - val_accuracy: 0.9630 - val_loss: 0.1095
Epoch 7/30
93/93 ━━━━━━━━━━━━━━━━━━━━ 5s 45ms/step - accuracy: 0.9730 - loss: 0.0862 - val_accuracy: 0.9805 - val_loss: 0.0636
Epoch 8/30
93/93 ━━━━━━━━━━━━━━━━━━━━ 4s 43ms/step - accuracy: 0.9771 - loss: 0.0710 - val_accuracy: 0.9845 - 

In [ ]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy:.4f}")


59/59 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9895 - loss: 0.0430
Test Accuracy: 0.9908


In [ ]:
from sklearn.metrics import classification_report

# Make predictions
y_pred = model.predict(X_test)

# Convert predictions to binary classes (0 or 1)
y_pred_classes = (y_pred > 0.5).astype(int)

# Print the classification report
print("Classification Report:")
print(classification_report(y_test, y_pred_classes))


59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.99      0.99       936
           1       0.99      0.99      0.99       921

    accuracy                           0.99      1857
   macro avg       0.99      0.99      0.99      1857
weighted avg       0.99      0.99      0.99      1857

